In [ ]:
import os
from tqdm.auto import tqdm
from google.colab import drive

import numpy as np
from sklearn.metrics import accuracy_score, f1_score, precision_score, recall_score
from sklearn.model_selection import train_test_split
from datasets import load_dataset, Dataset, DatasetDict
import torch
from transformers import (
    AutoTokenizer,
    AutoModelForSequenceClassification,
    TrainingArguments,
    Trainer,
    DataCollatorWithPadding
)

In [ ]:
MODEL_NAME = "microsoft/deberta-v3-base"
MAX_LENGTH = 512
BATCH_SIZE = 64
LEARNING_RATE = 2e-5
EPOCHS = 3
OUTPUT_DIR = '/content/drive/MyDrive/thesis/models/deberta/'

In [ ]:
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
data_filename = '/content/drive/MyDrive/thesis/data/depr_suic_google_gemma-2-9b-it_layer31_width16k.npz'
data_path = os.path.realpath(data_filename)
data = np.load(data_path, allow_pickle=True)
all_texts = data['texts'].tolist()
all_classes = data['classes'].tolist()
del data

In [ ]:
def load_data(all_texts, all_classes):
    texts_train, texts_test, y_train, y_test = train_test_split(
        all_texts,
        all_classes,
        test_size=2500,
        random_state=42,
        stratify=all_classes
    )

    texts_train, texts_val, y_train, y_val = train_test_split(
        texts_train,
        y_train,
        test_size=2500,
        random_state=42,
        stratify=y_train
    )

    train_dataset = Dataset.from_dict({
        'text': texts_train,
        'label': list(y_train)
    })

    val_dataset = Dataset.from_dict({
        'text': texts_val,
        'label': list(y_val)
    })

    test_dataset = Dataset.from_dict({
        'text': texts_test,
        'label': list(y_test)
    })

    dataset = DatasetDict({
        'train': train_dataset,
        'val': val_dataset,
        'test': test_dataset
    })

    return dataset

In [ ]:
tokenizer = AutoTokenizer.from_pretrained(MODEL_NAME)

/usr/local/lib/python3.12/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(
/usr/local/lib/python3.12/dist-packages/transformers/convert_slow_tokenizer.py:566: UserWarning: The sentencepiece tokenizer that you are converting to a fast tokenizer uses the byte fallback option which is not implemented in the fast tokenizers. In practice this means that the fast version of the tokenizer can produce unknown tokens whereas the sentencepiece version would have converted these unknown tokens into a sequence of byte tokens matching the original piece of text.
 

In [ ]:
def tokenize_function(examples):
    return tokenizer(
        examples['text'],
        truncation=True,
        max_length=MAX_LENGTH,
        padding=False
    )

In [ ]:
def compute_metrics(eval_pred):
    predictions, labels = eval_pred
    predictions = np.argmax(predictions, axis=1)

    accuracy = accuracy_score(labels, predictions)
    f1 = f1_score(labels, predictions, average='binary')
    precision = precision_score(labels, predictions, average='binary')
    recall = recall_score(labels, predictions, average='binary')

    return {
        'accuracy': accuracy,
        'f1': f1,
        'precision': precision,
        'recall': recall
    }

In [ ]:
all_classes = [1 if x == 'suicide' else 0 for x in all_classes]

In [ ]:
dataset = load_data(all_texts, all_classes)
dataset

DatasetDict({
    train: Dataset({
        features: ['text', 'label'],
        num_rows: 35000
    })
    val: Dataset({
        features: ['text', 'label'],
        num_rows: 2500
    })
    test: Dataset({
        features: ['text', 'label'],
        num_rows: 2500
    })
})

In [ ]:
tokenized_dataset = dataset.map(
    tokenize_function,
    batched=True,
    remove_columns=['text']
)

Map:   0%|          | 0/35000 [00:00<?, ? examples/s]

Map:   0%|          | 0/2500 [00:00<?, ? examples/s]

Map:   0%|          | 0/2500 [00:00<?, ? examples/s]

In [ ]:
model = AutoModelForSequenceClassification.from_pretrained(
    MODEL_NAME,
    num_labels=2
)

Some weights of DebertaV2ForSequenceClassification were not initialized from the model checkpoint at microsoft/deberta-v3-base and are newly initialized: ['classifier.bias', 'classifier.weight', 'pooler.dense.bias', 'pooler.dense.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [ ]:
data_collator = DataCollatorWithPadding(tokenizer=tokenizer)

In [ ]:
training_args = TrainingArguments(
    output_dir=OUTPUT_DIR,
    eval_strategy="epoch",
    save_strategy="epoch",
    save_total_limit=1,
    learning_rate=LEARNING_RATE,
    per_device_train_batch_size=BATCH_SIZE,
    per_device_eval_batch_size=BATCH_SIZE,
    num_train_epochs=EPOCHS,
    weight_decay=0.01,
    load_best_model_at_end=True,
    metric_for_best_model="f1",
    push_to_hub=False,
    logging_steps=100,
    fp16=torch.cuda.is_available(),
    dataloader_num_workers=4,
    gradient_accumulation_steps=1,
    report_to="none"
)

In [ ]:
trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=tokenized_dataset['train'],
    eval_dataset=tokenized_dataset['test'],
    tokenizer=tokenizer,
    data_collator=data_collator,
    compute_metrics=compute_metrics,
)

/tmp/ipython-input-142252713.py:1: FutureWarning: `tokenizer` is deprecated and will be removed in version 5.0.0 for `Trainer.__init__`. Use `processing_class` instead.
  trainer = Trainer(


In [ ]:
trainer.train()

The tokenizer has new PAD/BOS/EOS tokens that differ from the model config and generation config. The model config and generation config were aligned accordingly, being updated with the tokenizer's values. Updated tokens: {'eos_token_id': 2, 'bos_token_id': 1}.


Epoch,Training Loss,Validation Loss,Accuracy,F1,Precision,Recall
1,0.386300,0.379869,0.833200,0.838058,0.814340,0.863200
2,0.329800,0.362368,0.842000,0.846004,0.825095,0.868000
3,0.267500,0.382809,0.845200,0.847217,0.836321,0.858400


TrainOutput(global_step=1641, training_loss=0.3462833774155193, metrics={'train_runtime': 920.6332, 'train_samples_per_second': 114.052, 'train_steps_per_second': 1.782, 'total_flos': 2.76238673177856e+16, 'train_loss': 0.3462833774155193, 'epoch': 3.0})

In [ ]:
eval_results = trainer.evaluate(eval_dataset=tokenized_dataset['val'])
test_results = trainer.evaluate(eval_dataset=tokenized_dataset['test'])

In [ ]:
eval_results

{'eval_loss': 0.3751063942909241,
 'eval_accuracy': 0.8512,
 'eval_f1': 0.8516746411483254,
 'eval_precision': 0.848966613672496,
 'eval_recall': 0.8544,
 'eval_runtime': 8.0795,
 'eval_samples_per_second': 309.423,
 'eval_steps_per_second': 4.951,
 'epoch': 3.0}

In [ ]:
test_results

{'eval_loss': 0.38280943036079407,
 'eval_accuracy': 0.8452,
 'eval_f1': 0.8472167390446111,
 'eval_precision': 0.8363211223694466,
 'eval_recall': 0.8584,
 'eval_runtime': 8.0709,
 'eval_samples_per_second': 309.756,
 'eval_steps_per_second': 4.956,
 'epoch': 3.0}

In [ ]:
trainer.save_model(OUTPUT_DIR)